In [1]:
import pandas as pd

In [15]:
df = pd.DataFrame()

In [16]:
from logger import Logger

In [17]:
loggerDf = Logger('LOGS.csv', mode='df')

In [18]:
import difflib
def show_diff(text, n_text):
    """
    http://stackoverflow.com/a/788780
    Unify operations between two compared strings seqm is a difflib.
    SequenceMatcher instance whose a & b are strings
    """
    seqm = difflib.SequenceMatcher(None, text, n_text)
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<font color=red>^" + seqm.b[b0:b1] + "</font>")
        elif opcode == 'delete':
            output.append("<font color=blue>^" + seqm.a[a0:a1] + "</font>")
        elif opcode == 'replace':
            # seqm.a[a0:a1] -> seqm.b[b0:b1]
            output.append("<font color=green>^" + seqm.b[b0:b1] + "</font>")
        else:
            raise RuntimeError("unexpected opcode")
    return ''.join(output)

In [23]:
print(loggerDf.logsFile.loc[148]['trainloop'])

import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # Plotting
import seaborn as sns # Plotting

# Import Image Libraries - Pillow and OpenCV
from PIL import Image
import cv2

# Import PyTorch and useful fuctions
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
import torch.optim as optim
import torchvision.models as models # Pre-Trained models


# Import useful sklearn functions
import sklearn
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import time
from datetime import datetime
from tqdm import tqdm_notebook

import os
import random

# User-defined modules
from train_dataset import transforms_train, transforms_v

In [19]:
loggerDf.logsFile.loc[3, 'data-type'] = 'finetuning_new'
loggerDf.logsFile.loc[3, 'n-epochs'] = 50
loggerDf.logsFile.loc[3, 'batch-size'] = 16
loggerDf.logsFile.loc[3, 'time-estimated'] = 8100

,date,data-type,net-architecture,loss-func,optim,scheduler,early-stopping-patience,parameters-amount,n-epochs,batch-size,best-train-loss,best-valid-loss,best-kappa,lb-kappa-score,cfg,dataset,trainloop,time_estimated,time-estimated
0,2019.08.11 01:58:43,new_old_mixed_old,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.CosineAnnealingLR ob...,5.0,28342833.0,11.0,16.0,0.651234,0.631161,0.2751,NaN,from model import MainModel\nfrom torch import...,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,13907.77,NaN
1,2019.08.11 12:36:00,finetuning_new,import torch\nimport torch.nn as nn\nimport to...,MSELoss(),Adam (\nParameter Group 0\n amsgrad: False\...,<torch.optim.lr_scheduler.CosineAnnealingLR ob...,10.0,28342833.0,50.0,16.0,0.291845,0.305394,0.8427,NaN,from model import MainModel\nfrom torch import...,from torch.utils.data.dataset import Dataset\n...,import numpy as np # linear algebra\nimport pa...,4296.42,8100.0


In [23]:
loggerDf.save()

In [22]:
datetime.strftime(datetime.now(), "%Y.%m.%d %H:%M:%S")

'2019.08.02 00:51:32'

In [12]:
from libs.lr_finder import LRFinder


In [11]:
import torch

In [12]:
model = torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x16d_wsl')

Downloading: "https://github.com/facebookresearch/WSL-Images/archive/master.zip" to /home/paniquex/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x16-c6f796b0.pth" to /home/paniquex/.cache/torch/checkpoints/ig_resnext101_32x16-c6f796b0.pth
100%|██████████| 777518664/777518664 [01:08<00:00, 11411797.13it/s]


In [13]:
torch.save(model, 'resnext101_32x16d_wsl')

In [37]:
train_new = pd.read_csv('./input/train_new.csv')
train_old = pd.read_csv('./input/train_old.csv')

In [38]:
train_old

,id_code,diagnosis
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
5,15_right,2
6,16_left,4
7,16_right,4
8,17_left,0
9,17_right,1


In [36]:
train_old.to_csv('./input/train_old.csv', index=False)

,id_code,diagnosis
0,10_left,0
1,10_right,0
2,13_left,0
3,13_right,0
4,15_left,1
5,15_right,2
6,16_left,4
7,16_right,4
8,17_left,0
9,17_right,1
